In [ ]:
!pip install selenium
!pip install webdriver_manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Instalando o driver
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

# Abrindo a pagina desejada
driver.get('https://www.etf.com/etfanalytics/etf-finder')

# tempo para carregar o js
time.sleep(10)

# mudando a pagina para 100 elementos
botao_100 = driver.find_element("xpath",'/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span')

#botao_100.click()

## se der algum problema de click
driver.execute_script("arguments[0].click();", botao_100)

time.sleep(2)
numero_paginas = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]')
numero_paginas = numero_paginas.text.replace("of", "")
numero_paginas = int(numero_paginas)
numero_paginas


##
lista_tabela_por_pagina = []

elemento = driver.find_element("xpath",'/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div' )


for pagina in range(1, numero_paginas + 1):
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_proximo = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]')
    
    #botao_proximo.click()
    driver.execute_script("arguments[0].click();", botao_100)
    
    

tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)
tabela_cadastro_etfs


### voltano as paginas 
formulario_voltar_pagina = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/input')
formulario_voltar_pagina.clear()
formulario_voltar_pagina.send_keys('1')
formulario_voltar_pagina.send_keys(u'\ue007')

##
clicando_perfomace = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')
clicando_perfomace.click()


## A partir daqui é tudo igual

lista_tabela_por_pagina = []

elemento = driver.find_element("xpath",'/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div' )


for pagina in range(1, numero_paginas + 1):
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_proximo = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]')
    
    
    ## tivesso um bug no clicar igual o de antes
    driver.execute_script("arguments[0].click();", botao_proximo)
    #botao_proximo.click()
    
    

tabela_rentabilidade_etfs = pd.concat(lista_tabela_por_pagina)
tabela_rentabilidade_etfs

driver.quit()

## juntando as tabelas
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.set_index("Ticker")
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs[['1 Year', '3 Years', '5 Years']]
tabela_cadastro_etfs = tabela_cadastro_etfs.set_index("Ticker")

## juntando as tabelas numa só
base_de_dados_final = tabela_cadastro_etfs.join(tabela_rentabilidade_etfs, how='inner')
base_de_dados_final


In [ ]:
base_de_dados_final